# Battle of New York City Neigborhood Lifestyles

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          84 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  envi

## Download and Explore Dataset

Download New York Geo Data json (can also be found at https://geo.nyu.edu/catalog/nyu_2451_34572)

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


Load json

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Read features in new york data

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

Tranform the data into a *pandas* dataframe

In [12]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [13]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Lets read the data from neighborhoods data and write to pandas dataframe - neighborhoods

In [14]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [15]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [17]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


Lets create the map with these cordinates and superimpose neighborhoods on it

In [18]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Setting up Foursquare API and Credentials

In [173]:
CLIENT_ID = 'xx' # your Foursquare ID
CLIENT_SECRET = 'xx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xx
CLIENT_SECRET:xx


In [22]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

In [174]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=xx&client_secret=xx&v=20180605&ll=40.89470517661,-73.84720052054902&radius=500&limit=100'

Lets see if we can make calls

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c296065db04f53ab86f8633'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Wakefield', 'headerFullLocation': 'Wakefield, Bronx', 'headerLocationGranularity': 'neighborhood', 'totalResults': 7, 'suggestedBounds': {'ne': {'lat': 40.899205181110005, 'lng': -73.84125857127495}, 'sw': {'lat': 40.89020517211, 'lng': -73.8531424698231}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4c537892fd2ea593cb077a28', 'name': 'Lollipops Gelato', 'location': {'address': '4120 Baychester Ave', 'crossStreet': 'Edenwald & Bussing Ave', 'lat': 40.894123150205274, 'lng': -73.84589162362325, 'labeledLatLngs': [{'label': 'display', 'lat': 40.894123150205274, 'lng': -73.84589162362325}], 'distance': 127, 'postalC

## Venues in NYC Neighborhoods

Lets define a function to collect venues from returned response

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=900):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We define variable that will gather relavant information and put into 'nyc_venues' dataframe 

In [ ]:
nyc_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [33]:
print(nyc_venues.shape)
nyc_venues.head()

(18773, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
2,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy
4,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant


In [38]:
 nyc_venues.groupby(['Neighborhood']).count().reset_index().tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
297,Woodhaven,62,62,62,62,62,62
298,Woodlawn,47,47,47,47,47,47
299,Woodrow,19,19,19,19,19,19
300,Woodside,100,100,100,100,100,100
301,Yorkville,100,100,100,100,100,100


Save data incase - locally to avoid API calls when working with CC labs or IBM. They disconnect and you have to make calls again to retrieve data from Foursquare

In [48]:
nyc_venues.to_csv('nyc_venues.csv')

Assign new variable to work with relevant columns.

In [42]:
nyc_data = nyc_venues.filter(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'], axis=1)

In [43]:
nyc_data.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Wakefield,40.894705,-73.847201
1,Wakefield,40.894705,-73.847201
2,Wakefield,40.894705,-73.847201
3,Wakefield,40.894705,-73.847201
4,Wakefield,40.894705,-73.847201


Sort table

In [44]:
nyc_data.sort_values("Neighborhood", inplace = True)

Keep unique rows and clean up.

In [45]:
nyc_data.drop_duplicates(subset ="Neighborhood", inplace = True) 

Reset index

In [183]:
nyc_data.reset_index(drop=True).head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels
0,Allerton,40.865788,-73.859319,0
1,Annadale,40.538114,-74.178549,2
2,Arden Heights,40.549286,-74.185887,0
3,Arlington,40.635325,-74.165104,2
4,Arrochar,40.596313,-74.067124,1


Now we have all neighborhoods with their coordinates

In [47]:
nyc_data.shape

(302, 3)

## Removing irrelevant lifestyle venues

Lets look at the venues in neighborhoods again.

In [48]:
nyc_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
2,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy
4,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant


For our analysis of lifestyle we will remove food and transportation related categories

In [49]:
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Restaurant")]
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Pizza")]
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Deli")]
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Donut")]
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Diner")]
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Bank")]
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Hotel")]
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Intersection")]
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Bus")]
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Train")]
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Sandwich")]
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Joint")]
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].str.contains("Food")]

## Analyze neighborhoods

We will one hot encode each venue

In [50]:
# one hot encoding
nyc_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_onehot['Neighbourhood'] = nyc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[fixed_columns]

nyc_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport Lounge,Airport Service,Airport Terminal,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burrito Place,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Car Wash,Carpet Store,Check Cashing Service,Cheese Shop,Chocolate Shop,Christmas Market,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Stadium,College Theater,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cupcake Shop,Cycle Studio,Dance Studio,Daycare,Dentist's Office,Department Store,Design Studio,Dessert Shop,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Drugstore,Dry Cleaner,Duty-free Shop,Electronics Store,Elementary School,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Farm,Farmers Market,Field,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Football Stadium,Fountain,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hockey Rink,Home Service,Hookah Bar,Hostel,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Irish Pub,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Lake,Laser Tag,Laundromat,Laundry Service,Lawyer,Library,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Memorial Site,Men's Store,Metro Station,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Plane,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Art,Racetrack,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Scenic Lookout,School,Science Museum,Sculpture Garden,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Souvenir Shop,Souvlaki Shop,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stables,State / Provincial Park,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Supplement Shop,

In [51]:
nyc_onehot.shape

(10353, 348)

We calculate their frequency compared to other type of venues in each neighborhood by *mean*  and group them

In [182]:
nyc_grouped = nyc_onehot.groupby('Neighbourhood').mean().reset_index()
nyc_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport Lounge,Airport Service,Airport Terminal,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burrito Place,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Car Wash,Carpet Store,Check Cashing Service,Cheese Shop,Chocolate Shop,Christmas Market,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Stadium,College Theater,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cupcake Shop,Cycle Studio,Dance Studio,Daycare,Dentist's Office,Department Store,Design Studio,Dessert Shop,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Drugstore,Dry Cleaner,Duty-free Shop,Electronics Store,Elementary School,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Farm,Farmers Market,Field,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Football Stadium,Fountain,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hockey Rink,Home Service,Hookah Bar,Hostel,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Irish Pub,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Lake,Laser Tag,Laundromat,Laundry Service,Lawyer,Library,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Memorial Site,Men's Store,Metro Station,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Plane,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Art,Racetrack,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Scenic Lookout,School,Science Museum,Sculpture Garden,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Souvenir Shop,Souvlaki Shop,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stables,State / Provincial Park,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Supplement Shop,

In [79]:
nyc_grouped.shape

(302, 343)

In [ ]:
num_top_venues = 5

for hood in nyc_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = nyc_grouped[nyc_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Define function to sort common venues

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Targeting 20 most common venues based on their frequency in the neighborhood and creating dataframe

In [181]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nyc_grouped['Neighbourhood']

for ind in np.arange(nyc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Allerton,Pharmacy,Supermarket,Dessert Shop,Brewery,Bar,Metro Station,Martial Arts Dojo,Smoke Shop,Spa,Basketball Court,Playground,Mobile Phone Shop,Clothing Store,Department Store,Dance Studio,Cosmetics Shop,Breakfast Spot,Shoe Store,Farmers Market,Farm
1,Annadale,Trail,Sports Bar,Bakery,Home Service,Elementary School,Liquor Store,Park,Dessert Shop,Fish Market,Farmers Market,Event Space,Fountain,Exhibit,Fabric Shop,Factory,Farm,Field,Fishing Store,Football Stadium,Film Studio
2,Arden Heights,Gift Shop,Liquor Store,Pharmacy,Shopping Mall,Moving Target,Coffee Shop,Spa,Elementary School,Lawyer,Fish & Chips Shop,Farm,Exhibit,Football Stadium,Fabric Shop,Flower Shop,Flea Market,Factory,Fishing Store,Financial or Legal Service,Farmers Market
3,Arlington,Rental Service,Boat or Ferry,Automotive Shop,Coffee Shop,Nightlife Spot,Snack Place,Fountain,Football Stadium,Exhibit,Fabric Shop,Fruit & Vegetable Store,Factory,Farm,Farmers Market,Field,Frozen Yogurt Shop,Film Studio,Financial or Legal Service,Event Space,Fish Market
4,Arrochar,Beach,Baseball Field,Supermarket,Bagel Shop,Juice Bar,Dance Studio,Department Store,Playground,Sculpture Garden,Park,Steakhouse,Taco Place,Moving Target,Toll Plaza,Cosmetics Shop,Ice Cream Shop,History Museum,Farm,Electronics Store,Elementary School


In [163]:
# set number of clusters
kclusters = 4

nyc_grouped_clustering = nyc_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 2, 1, 1, 2, 0, 0, 2], dtype=int32)

Verify two dataframe shapes to avoid error in merging

In [164]:
kmeans.labels_.shape

(302,)

In [165]:
nyc_data.shape

(302, 4)

Since they are of same size and sorted ascending we merge nyc_data table consists of neighborhood lat, long with 20 most commmon places dataset neighborhoods_venues_sorted adding cluster labels.

In [166]:
nyc_merged = nyc_data

# add clustering labels
nyc_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
nyc_merged = nyc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

nyc_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
18378,Allerton,40.865788,-73.859319,0,Pharmacy,Supermarket,Dessert Shop,Brewery,Bar,Metro Station,Martial Arts Dojo,Smoke Shop,Spa,Basketball Court,Playground,Mobile Phone Shop,Clothing Store,Department Store,Dance Studio,Cosmetics Shop,Breakfast Spot,Shoe Store,Farmers Market,Farm
14226,Annadale,40.538114,-74.178549,2,Trail,Sports Bar,Bakery,Home Service,Elementary School,Liquor Store,Park,Dessert Shop,Fish Market,Farmers Market,Event Space,Fountain,Exhibit,Fabric Shop,Factory,Farm,Field,Fishing Store,Football Stadium,Film Studio
15188,Arden Heights,40.549286,-74.185887,0,Gift Shop,Liquor Store,Pharmacy,Shopping Mall,Moving Target,Coffee Shop,Spa,Elementary School,Lawyer,Fish & Chips Shop,Farm,Exhibit,Football Stadium,Fabric Shop,Flower Shop,Flea Market,Factory,Fishing Store,Financial or Legal Service,Farmers Market
14717,Arlington,40.635325,-74.165104,2,Rental Service,Boat or Ferry,Automotive Shop,Coffee Shop,Nightlife Spot,Snack Place,Fountain,Football Stadium,Exhibit,Fabric Shop,Fruit & Vegetable Store,Factory,Farm,Farmers Market,Field,Frozen Yogurt Shop,Film Studio,Financial or Legal Service,Event Space,Fish Market
14740,Arrochar,40.596313,-74.067124,1,Beach,Baseball Field,Supermarket,Bagel Shop,Juice Bar,Dance Studio,Department Store,Playground,Sculpture Garden,Park,Steakhouse,Taco Place,Moving Target,Toll Plaza,Cosmetics Shop,Ice Cream Shop,History Museum,Farm,Electronics Store,Elementary School


In [167]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_merged['Neighborhood Latitude'], nyc_merged['Neighborhood Longitude'], nyc_merged['Neighborhood'], nyc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 1st CLUSTER

The Technician neighborhood

In [177]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 0, nyc_merged.columns[[1] + list(range(0, nyc_merged.shape[1]))]].head()

,Neighborhood Latitude,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
18378,40.865788,Allerton,40.865788,-73.859319,0,Pharmacy,Supermarket,Dessert Shop,Brewery,Bar,Metro Station,Martial Arts Dojo,Smoke Shop,Spa,Basketball Court,Playground,Mobile Phone Shop,Clothing Store,Department Store,Dance Studio,Cosmetics Shop,Breakfast Spot,Shoe Store,Farmers Market,Farm
15188,40.549286,Arden Heights,40.549286,-74.185887,0,Gift Shop,Liquor Store,Pharmacy,Shopping Mall,Moving Target,Coffee Shop,Spa,Elementary School,Lawyer,Fish & Chips Shop,Farm,Exhibit,Football Stadium,Fabric Shop,Flower Shop,Flea Market,Factory,Fishing Store,Financial or Legal Service,Farmers Market
16418,40.770317,Astoria Heights,40.770317,-73.894680,0,Rental Car Location,Airport Terminal,Bakery,Discount Store,Plaza,Bar,Baseball Field,Men's Store,Gym / Fitness Center,Gym,Grocery Store,Bowling Alley,Furniture / Home Store,Flower Shop,Hardware Store,Café,Park,Pet Store,Pharmacy,Rock Climbing Spot
11477,40.761730,Auburndale,40.761730,-73.791762,0,Cosmetics Shop,Mobile Phone Shop,Ice Cream Shop,Café,Pet Store,Pharmacy,Gym / Fitness Center,Gym,Park,Supermarket,Bar,Toy / Game Store,Electronics Store,Mattress Store,Movie Theater,Miscellaneous Shop,Taco Place,Lounge,Liquor Store,Noodle House
12476,40.782843,Bay Terrace,40.782843,-73.776802,0,Women's Store,Clothing Store,Supermarket,Cosmetics Shop,Lingerie Store,Gym,Mobile Phone Shop,Kids Store,Harbor / Marina,Furniture / Home Store,Men's Store,Gift Shop,Movie Theater,Sporting Goods Shop,Steakhouse,Shopping Mall,Performing Arts Venue,Shipping Store,Pharmacy,Salon / Barbershop


#### 2nd CLUSTER

The Adventurer’s neighborhood

In [178]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 1, nyc_merged.columns[[1] + list(range(0, nyc_merged.shape[1]))]].head()

,Neighborhood Latitude,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
14740,40.596313,Arrochar,40.596313,-74.067124,1,Beach,Baseball Field,Supermarket,Bagel Shop,Juice Bar,Dance Studio,Department Store,Playground,Sculpture Garden,Park,Steakhouse,Taco Place,Moving Target,Toll Plaza,Cosmetics Shop,Ice Cream Shop,History Museum,Farm,Electronics Store,Elementary School
12555,40.589144,Arverne,40.589144,-73.791992,1,Surf Spot,Beach,Metro Station,Supermarket,Playground,Building,Boat or Ferry,Taco Place,Board Shop,Grocery Store,Bed & Breakfast,Gym / Fitness Center,Brewery,Ice Cream Shop,Dive Bar,Dentist's Office,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store
18650,40.611322,Bayswater,40.611322,-73.765968,1,Playground,Beach,Park,Home Service,Athletics & Sports,Zoo Exhibit,Field,Exhibit,Fabric Shop,Factory,Farm,Farmers Market,Film Studio,Event Service,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop
13282,40.576156,Belle Harbor,40.576156,-73.854018,1,Beach,Boutique,Spa,Event Space,Bakery,Bagel Shop,Harbor / Marina,Smoke Shop,Pub,Flea Market,Factory,Fruit & Vegetable Store,Exhibit,Frozen Yogurt Shop,Fountain,Fabric Shop,Football Stadium,Farm,Fishing Store,Farmers Market
6066,40.615150,Bergen Beach,40.615150,-73.898556,1,Harbor / Marina,Stables,Gym,Playground,Athletics & Sports,Baseball Field,Zoo Exhibit,Field,Exhibit,Fabric Shop,Factory,Farm,Farmers Market,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop


#### 3rd CLUSTER

The Socialite’s neighborhood 

In [179]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 2, nyc_merged.columns[[1] + list(range(0, nyc_merged.shape[1]))]].head()

,Neighborhood Latitude,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
14226,40.538114,Annadale,40.538114,-74.178549,2,Trail,Sports Bar,Bakery,Home Service,Elementary School,Liquor Store,Park,Dessert Shop,Fish Market,Farmers Market,Event Space,Fountain,Exhibit,Fabric Shop,Factory,Farm,Field,Fishing Store,Football Stadium,Film Studio
14717,40.635325,Arlington,40.635325,-74.165104,2,Rental Service,Boat or Ferry,Automotive Shop,Coffee Shop,Nightlife Spot,Snack Place,Fountain,Football Stadium,Exhibit,Fabric Shop,Fruit & Vegetable Store,Factory,Farm,Farmers Market,Field,Frozen Yogurt Shop,Film Studio,Financial or Legal Service,Event Space,Fish Market
9704,40.768509,Astoria,40.768509,-73.915654,2,Bar,Bakery,Grocery Store,Pub,Coffee Shop,Hookah Bar,Gourmet Shop,Health & Beauty Service,Bagel Shop,Gym,Salon / Barbershop,Ice Cream Shop,Martial Arts Dojo,Beer Garden,Salad Place,Basketball Court,Souvlaki Shop,Gym / Fitness Center,Spa,Brewery
5275,40.599519,Bath Beach,40.599519,-73.998752,2,Bakery,Coffee Shop,Supplement Shop,Dessert Shop,Pharmacy,Bubble Tea Shop,Gourmet Shop,Mobile Phone Shop,Kids Store,Gift Shop,Electronics Store,Park,Tea Room,Surf Spot,Gym,Gym / Fitness Center,Frozen Yogurt Shop,Shoe Store,Parking,Spa
9484,40.711932,Battery Park City,40.711932,-74.016869,2,Coffee Shop,Park,Wine Shop,Plaza,Gym,Ice Cream Shop,Kids Store,Bookstore,Market,Grocery Store,Steakhouse,Gym / Fitness Center,Fountain,Scenic Lookout,Shopping Mall,Performing Arts Venue,Dog Run,Furniture / Home Store,Playground,Cupcake Shop


#### 4th CLUSTER

The Provider’s neighborhood 

In [180]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 3, nyc_merged.columns[[1] + list(range(0, nyc_merged.shape[1]))]].head()

,Neighborhood Latitude,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
13371,40.660003,Brookville,40.660003,-73.751753,3,Park,Lake,Bakery,Laundromat,Clothing Store,Gym / Fitness Center,Gym,Grocery Store,Film Studio,Factory,Farm,Farmers Market,Field,Financial or Legal Service,Exhibit,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop
16437,40.831428,Claremont Village,40.831428,-73.901199,3,Discount Store,Ice Cream Shop,Gym,Park,Zoo Exhibit,Exhibit,Fabric Shop,Factory,Farm,Farmers Market,Field,Film Studio,Financial or Legal Service,Event Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Football Stadium
1580,40.806551,Clason Point,40.806551,-73.854144,3,Park,Discount Store,Boat or Ferry,Scenic Lookout,Zoo Exhibit,Exhibit,Fabric Shop,Factory,Farm,Farmers Market,Field,Film Studio,Financial or Legal Service,Fish Market,Fish & Chips Shop,Event Service,Fishing Store,Flea Market,Flower Shop,Football Stadium
4881,40.669926,East New York,40.669926,-73.880699,3,Yoga Studio,Grocery Store,Park,Video Game Store,Metro Station,Health & Beauty Service,Construction & Landscaping,Check Cashing Service,Art Gallery,Playground,Field,Exhibit,Fabric Shop,Factory,Farm,Farmers Market,Financial or Legal Service,Film Studio,Event Service,Fish & Chips Shop
12511,40.595642,Edgemere,40.595642,-73.776133,3,Park,Metro Station,Supermarket,Skate Park,Beach,Grocery Store,Farm,Gift Shop,Scenic Lookout,Spa,Film Studio,Financial or Legal Service,Event Service,Fish & Chips Shop,Field,Farmers Market,Fish Market,Factory,Fishing Store,Flea Market
